In [2]:
import os
import torch
from options import Option
from data_utils.dataset import load_data_test
from model.model import Model
from utils.util import setup_seed, load_checkpoint
import torchvision
import einops

In [3]:
args = Option().parse()
args.load = "./checkpoint/sketchy_ext/best_checkpoint.pth"
args.batch = 2

print("test args:", str(args))

test args: Namespace(data_path='./datasets', dataset='sketchy_extend', test_class='test_class_sketchy25', cls_number=100, d_model=768, d_ff=1024, head=8, number=1, pretrained=True, anchor_number=49, save='./checkpoints/sketchy_ext', batch=2, epoch=30, datasetLen=10000, learning_rate=1e-05, weight_decay=0.01, load='./checkpoint/sketchy_ext/best_checkpoint.pth', retrieval='rn', testall=False, test_sk=20, test_im=20, num_workers=4, choose_cuda='0', seed=2021)


In [4]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from utils.ap import calculate
from tqdm import tqdm
import time

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = args.choose_cuda
print("current cuda: " + args.choose_cuda)
setup_seed(args.seed)

current cuda: 0


In [6]:
# prepare data
sk_valid_data, im_valid_data = load_data_test(args)

# prepare model
model = Model(args)
model = model.half()

if args.load is not None:
    checkpoint = load_checkpoint(args.load)

cur = model.state_dict()
new = {k: v for k, v in checkpoint['model'].items() if k in cur.keys()}
cur.update(new)
model.load_state_dict(cur)

if len(args.choose_cuda) > 1:
    model = torch.nn.parallel.DataParallel(model.to('cuda'))
model = model.cuda()

used for valid or test sketch / image:
(77,) (1711,)
used for train sketch / image:
(55252,) (68401,)
=> loading model './checkpoint/sketchy_ext/best_checkpoint.pth'


In [7]:
model.eval()
torch.set_grad_enabled(False)

print('loading image data')
sk_dataload = DataLoader(sk_valid_data, batch_size=args.test_sk, num_workers=args.num_workers, drop_last=False)
print('loading sketch data')
im_dataload = DataLoader(im_valid_data, batch_size=args.test_im, num_workers=args.num_workers, drop_last=False)

dist_im = None
all_dist = None

loading image data
loading sketch data


In [8]:
for i, (sk, sk_label) in enumerate(tqdm(sk_dataload)):
        #sk.shape=(20,3,224,224)
        print(i)
        if i == 0:
            all_sk_label = sk_label.numpy()
        else:
            all_sk_label = np.concatenate((all_sk_label, sk_label.numpy()), axis=0)

        sk_len = sk.size(0)
        sk = sk.cuda()
        #debug
        print(sk[0].shape)
        # cv2.imwrite(f"./output/sk-{i}",sk[0].cpu().numpy())
        if i==0:
            grid_sk = torchvision.utils.make_grid(sk)
            torchvision.utils.save_image(grid_sk,f"./output/sk.jpg")
        
        sk, sk_idxs = model(sk, None, 'test', only_sa=True)#sk.shape=(20,192,768)
        for j, (im, im_label) in enumerate(tqdm(im_dataload)):
            if i == 0 and j == 0:
                all_im_label = im_label.numpy()
            elif i == 0 and j > 0:
                all_im_label = np.concatenate((all_im_label, im_label.numpy()), axis=0)

            im_len = im.size(0)
            im = im.cuda()
            im, im_idxs = model(im, None, 'test', only_sa=True)

            sk_temp = sk.unsqueeze(1).repeat(1, im_len, 1, 1).flatten(0, 1).cuda() #(400,197,768) #?difference
            im_temp = im.unsqueeze(0).repeat(sk_len, 1, 1, 1).flatten(0, 1).cuda() #(400,197,768)
            
            if args.retrieval == 'rn':
                feature_1, feature_2 = model(sk_temp, im_temp, 'test')
            #? when retrieval == 'sa'
            if args.retrieval == 'sa':
                feature_1, feature_2 = torch.cat((sk_temp[:, 0], im_temp[:, 0]), dim=0), None

            # print(feature_1.size())    # [2*sk*im, 768] #2 means sk and im cls
            # print(feature_2.size())    # [sk*im, 1]

            if args.retrieval == 'rn':
                if j == 0:
                    dist_im = - feature_2.view(sk_len, im_len).cpu().data.numpy()  # 1*args.batch
                else:
                    dist_im = np.concatenate((dist_im, - feature_2.view(sk_len, im_len).cpu().data.numpy()), axis=1)
            if args.retrieval == 'sa':
                dist_temp = F.pairwise_distance(F.normalize(feature_1[:sk_len * im_len]),
                                                F.normalize(feature_1[sk_len * im_len:]), 2)
                if j == 0:
                    dist_im = dist_temp.view(sk_len, im_len).cpu().data.numpy()
                else:
                    dist_im = np.concatenate((dist_im, dist_temp.view(sk_len, im_len).cpu().data.numpy()), axis=1)

        if i == 0:
            all_dist = dist_im
        else:
            all_dist = np.concatenate((all_dist, dist_im), axis=0)
        print(all_dist.shape)
        #all_dist.shape=(all_sk_label.size, all_im_label.size)
    # print(all_sk_label.size, all_im_label.size)     # [762 x 1711] / 2
class_same = (np.expand_dims(all_sk_label, axis=1) == np.expand_dims(all_im_label, axis=0)) * 1
# print(all_dist.size, class_same.size)     # [762 x 1711] / 2


  0%|          | 0/4 [00:00<?, ?it/s]

0
torch.Size([3, 224, 224])


 25%|██▌       | 1/4 [00:10<00:32, 10.97s/it]

(20, 1711)
1
torch.Size([3, 224, 224])


 50%|█████     | 2/4 [00:18<00:18,  9.12s/it]

(40, 1711)
2
torch.Size([3, 224, 224])


 75%|███████▌  | 3/4 [00:26<00:08,  8.54s/it]

(60, 1711)
3
torch.Size([3, 224, 224])


100%|██████████| 4/4 [00:33<00:00,  8.42s/it]

(77, 1711)


In [9]:
print(class_same.shape)
print(class_same)
np.savetxt("./output/all_dist",all_dist)
np.savetxt("./output/class_same",class_same)


(77, 1711)
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]


In [10]:
map_all, map_200, precision100, precision200 = calculate(all_dist, class_same, test=True)
print(map_all,map_200,precision100,precision200)

eval time: 0.5300471782684326
0.680195628754997 0.6593741691830883 0.6408593850189362 0.6408593850189362


In [11]:
arg_sort_sim = all_dist.argsort()   # 得到从小到大索引值
print(arg_sort_sim.shape)
print(arg_sort_sim)
np.savetxt("./output/arg_sort_sim",torch.tensor(arg_sort_sim,dtype=int))

(77, 1711)
[[ 286  254  302 ...  173 1272  179]
 [ 382  302  271 ...  342  349  347]
 [ 253  302  255 ...  349  228  179]
 ...
 [ 160 1244 1213 ...  582 1366  604]
 [1411 1455 1421 ...  337 1213  567]
 [ 266  971  247 ... 1304  928  819]]


In [12]:
def patch_replace_data(sk_index, im_index):
    '''
    sk_index: 0
    im_index: list
    '''
    (sk,_) = sk_valid_data[sk_index]
    sk = torch.unsqueeze(sk,0)
    
    for i,v in enumerate(im_index):
        if i == 0:    
            (im,_) = im_valid_data[im_index[0]]
            im = torch.unsqueeze(im,0)
        else:    
            im = torch.cat((im,im_valid_data[v][0].unsqueeze(0)))
    
    return sk,im
    

In [13]:
sk_index=0
im_index = [286 , 254 , 302]
(sk_tmp, im_tmp) = patch_replace_data(sk_index,im_index)
print(sk_tmp.shape, im_tmp.shape)

torchvision.utils.save_image(sk_tmp.cuda(),f"./output/sk-{sk_index}.jpg")

im_tmp = torchvision.utils.make_grid(im_tmp)
torchvision.utils.save_image(im_tmp.cuda(),f"./output/im_top_{len(im_index)}.jpg")
print(sk_tmp.shape, im_tmp.shape)

torch.Size([1, 3, 224, 224]) torch.Size([3, 3, 224, 224])
torch.Size([1, 3, 224, 224]) torch.Size([3, 228, 680])


In [14]:
from model import rn

sk, im = patch_replace_data(sk_index, im_index)
print(sk.shape, im.shape)


sk_sa, sk_idxs = model(sk.cuda(), None, 'test', only_sa=True)#sk_sa.shape=(20,192,768)
im_sa, im_idxs = model(im.cuda(), None, 'test', only_sa=True)#im_sa.shape=(20,192,768)


sk_im_sa = torch.cat((sk_sa, im_sa), dim=0)
ca_fea = model.ca(sk_im_sa)  # [2b, 197, 768]
cls_fea = ca_fea[:, 0]  # [2b, 1, 768]
token_fea = ca_fea[:, 1:]  # [2b, 196, 768]
batch = token_fea.size(0)

print(token_fea.shape)

torch.Size([1, 3, 224, 224]) torch.Size([3, 3, 224, 224])
torch.Size([4, 196, 768])


In [15]:
# token_fea = einops.rearrange(token_fea,"b d h w -> b d (h w)") #token_fea = token_fea.view(batch, 768, 14, 14)
sk_fea = token_fea[0]
im_fea = token_fea[sk.size(0):]
# np.savetxt("./output/sk_fea", sk_fea.cpu())
# np.savetxt("./output/im_fea", im_fea.cpu())
print(sk_fea.shape, im_fea.shape)
cos_scores = rn.cos_similar(sk_fea, im_fea)
print(cos_scores.shape)
np.savetxt("./output/cos_scores",cos_scores.cpu()[0])

torch.Size([196, 768]) torch.Size([3, 196, 768])
torch.Size([3, 196, 196])


In [16]:
# print(cos_scores.argsort(0).shape,cos_scores.argsort(0))
# print(torch.argmax(einops.rearrange(cos_scores,"a b c -> b (a c)")))
b = einops.rearrange(cos_scores,"a b c -> b (a c)")
# print(cos_scores.shape,cos_scores)

max_indices = torch.empty((0,2), dtype=int)
print(b)
print(max_indices)

for i in b:
    max_indices_item = torch.argmax(i)
    # print(i.shape)
    new = np.unravel_index(max_indices_item.cpu(),(cos_scores.shape[0],cos_scores.shape[2]))
    # print(torch.Tensor(new))
    max_indices = torch.cat((max_indices, torch.tensor(new, dtype=torch.int).unsqueeze(0)), 0)
    # print(max_indices)
    
# print(np.unravel_index(b.values, (3, 196)))
np.savetxt("./output/max_indices",max_indices)


tensor([[0.8413, 0.8198, 0.8320,  ..., 0.6865, 0.6904, 0.6870],
        [0.8403, 0.8306, 0.8423,  ..., 0.6860, 0.6885, 0.6831],
        [0.8345, 0.8306, 0.8447,  ..., 0.6841, 0.6855, 0.6772],
        ...,
        [0.8340, 0.8271, 0.8389,  ..., 0.6875, 0.6870, 0.6826],
        [0.8350, 0.8218, 0.8345,  ..., 0.6914, 0.6934, 0.6899],
        [0.8359, 0.8115, 0.8247,  ..., 0.6899, 0.6929, 0.6948]],
       device='cuda:0', dtype=torch.float16)
tensor([], size=(0, 2), dtype=torch.int64)


In [17]:
def patch2im(patch_index,im, patch_size):
    '''
    im: (c, w, h)
    patch_index: (2)
    return: (c, patch_size, patch_size)
    '''
    # print(patch_index.shape, im.shape, patch_size)
    # print(patch_index)
    # print(patch_index[0].item()*patch_size)
        
    return im[:, \
        patch_index[0]*patch_size:(patch_index[0].item()+1)*patch_size, \
        patch_index[1].item()*patch_size:(patch_index[1].item()+1)*patch_size]


In [18]:
def patch_match(im, indices,patch_size):
    '''
        im: (b,c,w,h)
        indices: (m,im.shape.len)
    '''
    # print(im.shape)
    x = torch.zeros((0,)+tuple(im.shape[1:]))
    # print(x)
    for i in indices:
        patch_index = np.unravel_index(i[1],(im.size(-1)/patch_size,im.size(-1)/patch_size))
        item = patch2im(torch.tensor(patch_index,dtype=int), im[i[0]], patch_size)
        x= torch.cat([x, item])
    return x 


In [19]:
#patch replace op test
# indices = max_indices

# print(im.shape)
# # x = torch.zeros((0,)+tuple(im.shape[1:]))
# x = torch.zeros((0, 3, 14,14))
# # print(x)
# for i,v in enumerate(indices):
#     patch_index = np.unravel_index(i,(16,16))
#     item = patch2im(torch.tensor(patch_index,dtype=int), im[0], im.shape[-1]//16)
#     # print(item.shape)
#     x= torch.cat([x, item.unsqueeze(0)])


In [20]:
indices = max_indices

print(im.shape)
# x = torch.zeros((0,)+tuple(im.shape[1:]))
x = torch.zeros((0, 3, 16,16))
# print(x)
for i in indices:
    patch_index = np.unravel_index(i[1],(14,14))
    item = patch2im(torch.tensor(patch_index,dtype=int), im[i[0]], int(im.shape[-1]/14))
    # print(item.shape)
    x= torch.cat([x, item.unsqueeze(0)])

torch.Size([3, 3, 224, 224])


In [21]:
x = torchvision.utils.make_grid(x,nrow=14)
torchvision.utils.save_image(x,"./output/patch_replace.jpg")

In [22]:
print(x.shape,x)

torch.Size([3, 254, 254]) tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.1059,  ...,  0.1843,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000, -0.1294,  ..., -0.0510,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.3882,  ..., -0.3569,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000, -0.5059,  ..., -0.4980,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, 

In [23]:
# valid
# map_all, map_200, precision_100, precision_200 = valid_cls(args, model, sk_valid_data, im_valid_data)
print(f'map_all:{map_all:.4f} map_200:{map_200:.4f} precision_100:{precision100:.4f} precision_200:{precision200:.4f}')

map_all:0.6802 map_200:0.6594 precision_100:0.6409 precision_200:0.6409


In [ ]:
vqgan_dict = torch.load("../download/last.ckpt")
print(vqgan_dict)

In [27]:
import sys 

sys.path.append("./vqgan_pytorch/")

import vqgan

In [28]:
import argparse
parser = argparse.ArgumentParser(description="VQGAN")
parser.add_argument('--latent-dim', type=int, default=256, help='Latent dimension n_z (default: 256)')
parser.add_argument('--image-size', type=int, default=256, help='Image height and width (default: 256)')
parser.add_argument('--num-codebook-vectors', type=int, default=1024, help='Number of codebook vectors (default: 256)')
parser.add_argument('--beta', type=float, default=0.25, help='Commitment loss scalar (default: 0.25)')
parser.add_argument('--image-channels', type=int, default=3, help='Number of channels of images (default: 3)')
parser.add_argument('--dataset-path', type=str, default='/data', help='Path to data (default: /data)')
parser.add_argument('--device', type=str, default="cuda", help='Which device the training is on')
parser.add_argument('--batch-size', type=int, default=6, help='Input batch size for training (default: 6)')
parser.add_argument('--epochs', type=int, default=100, help='Number of epochs to train (default: 50)')
parser.add_argument('--learning-rate', type=float, default=2.25e-05, help='Learning rate (default: 0.0002)')
parser.add_argument('--beta1', type=float, default=0.5, help='Adam beta param (default: 0.0)')
parser.add_argument('--beta2', type=float, default=0.9, help='Adam beta param (default: 0.999)')
parser.add_argument('--disc-start', type=int, default=10000, help='When to start the discriminator (default: 0)')
parser.add_argument('--disc-factor', type=float, default=1., help='')
parser.add_argument('--rec-loss-factor', type=float, default=1., help='Weighting factor for reconstruction loss.')
parser.add_argument('--perceptual-loss-factor', type=float, default=1., help='Weighting factor for perceptual loss.')

args2 = parser.parse_args([])

In [31]:
vqgan_model = vqgan.VQGAN(args2)
print(vqgan_model)
vqgan_model.load_state_dict(vqgan_dict[state_dict])

VQGAN(
  (encoder): Encoder(
    (model): Sequential(
      (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ResidualBlock(
        (block): Sequential(
          (0): GroupNorm(
            (gn): GroupNorm(32, 128, eps=1e-06, affine=True)
          )
          (1): Swish()
          (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): GroupNorm(
            (gn): GroupNorm(32, 128, eps=1e-06, affine=True)
          )
          (4): Swish()
          (5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (2): ResidualBlock(
        (block): Sequential(
          (0): GroupNorm(
            (gn): GroupNorm(32, 128, eps=1e-06, affine=True)
          )
          (1): Swish()
          (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): GroupNorm(
            (gn): GroupNorm(32, 128, eps=1e-06, affine=True)
          )
          (4): Swish()
    

AttributeError: 'dict' object has no attribute 'state_dict'

In [25]:
print(vqgan_model)

NameError: name 'vqgan_model' is not defined

In [28]:
print(model)

Model(
  (sa): Self_Attention(
    (model): ViTPatch(
      (embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (scale): Scale_Embedding(
        (seq): Sequential(
          (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (1): ReLU(inplace=True)
          (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (3): ReLU(inplace=True)
          (4): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (5): ReLU(inplace=True)
          (6): Conv2d(256, 768, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        )
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (transformer): Encoder(
        (encoder_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0-11): 12 x ModuleList(
            (0): Encoder1DBlock(
              (layer_norm_input): LayerNorm((768,), eps=1e-05, el